In [1]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkContext
sc =SparkContext()


23/04/29 20:36:35 WARN Utils: Your hostname, codys-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.250.240.97 instead (on interface en0)
23/04/29 20:36:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/29 20:36:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load and parse the data

In [2]:

data = sc.textFile("/Users/codyyu/Desktop/CMPE 256/steam/data/final.csv")


In [3]:
data.glom().collect()

[['76561197970982479,1250,1,76561197970982479,10006,0,0',
  '76561197970982479,22200,1,76561197970982479,271,0,0',
  '76561197970982479,43110,1,76561197970982479,834,0,0',
  'js41637,227300,1,76561198035864385,551,0,1',
  'js41637,239030,1,76561198035864385,349,0,1',
  'evcentric,248820,1,76561198007712555,2328,0,2',
  'evcentric,237930,1,76561198007712555,508,0,2',
  'evcentric,107200,1,76561198007712555,3225,0,2',
  'doctr,250320,1,76561198002099482,778,0,3',
  'doctr,20920,1,76561198002099482,2981,0,3',
  'doctr,204100,1,76561198002099482,5297,0,3',
  'doctr,207610,1,76561198002099482,1972,0,3',
  'doctr,108710,1,76561198002099482,1185,0,3',
  'maplemage,211420,1,76561198026584251,12347,0,4',
  'maplemage,211820,1,76561198026584251,3436,0,4',
  'maplemage,730,1,76561198026584251,13955,0,4',
  'maplemage,204300,1,76561198026584251,17589,0,4',
  'MeaTCompany,730,1,76561198071532177,18481,507,5',
  '76561198089393905,72850,1,76561198089393905,37776,16,6',
  '76561198156664158,252950,1,

In [4]:
data.take(11)[0]

'76561197970982479,1250,1,76561197970982479,10006,0,0'

In [5]:
ratings = data.map(lambda l: l.split(','))\
    .map(lambda l: Rating(int(l[6]), int(l[1]), float(l[2])))

In [6]:
ratings.take(1)

[Rating(user=0, product=1250, rating=1.0)]

In [7]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Build the recommendation model using Alternating Least Squares


In [8]:

rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

23/04/29 20:36:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/29 20:36:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


## Evaluate the model on training data

In [9]:

testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))



Mean Squared Error = 0.00224787965230073


In [11]:
# Save and load model
model.save(sc, "model/ALSCollaborativeFilter")
sameModel = MatrixFactorizationModel.load(sc, "model/ALSCollaborativeFilter")

23/04/29 20:39:01 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 20:39:01 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/29 20:39:01 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/04/29 20:39:01 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/04/29 20:39:01 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/04/29 20:39:01 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers
23/04/29 20:39:02 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,

In [12]:
import csv

reader = csv.reader(open('/Users/codyyu/Desktop/CMPE 256/steam/data/item_id_game_name_map.csv', 'r'))
game_map = {}
for row in reader:
   k, v = row
   game_map[k] = v

In [13]:
model.predict(0, 730)

0.41004564348590833

In [14]:
sameModel.recommendProducts(1, 2)

[Rating(user=1, product=110800, rating=1.2631208871169841),
 Rating(user=1, product=233150, rating=1.1833107519733348)]

In [15]:
# Build the recommendation model using Alternating Least Squares based on implicit ratings
model = ALS.trainImplicit(ratings, rank, numIterations, alpha=0.01)

In [16]:
model.predict(21, 359320)

0.00085881690794882

In [ ]:
model.recommendProducts(1, 2)

[Rating(user=1, product=72850, rating=0.05762378146105467),
 Rating(user=1, product=105600, rating=0.018450447627698895)]

23/04/30 00:07:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3889020 ms exceeds timeout 120000 ms
23/04/30 00:07:46 WARN SparkContext: Killing executors is not supported by current scheduler.
23/04/30 00:07:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.